# Imports

In [ ]:
import csv
import io
import numpy as np
import os
from zipfile import ZipFile
import warnings
import re
import xml.etree.ElementTree as ET
import requests


warnings.filterwarnings("ignore")

# Collect

In [ ]:
path = 'https://raw.githubusercontent.com/luofuli/word-sense-disambiguation/master/data/All_Words_WSD/Evaluation_Datasets/semeval2015/semeval2015.data.xml'
download = requests.get(path).content

In [ ]:
tree = ET.parse(io.StringIO(download.decode('utf-8')))
root = tree.getroot()

In [ ]:
#accessing the xml file
lemma_sentence_dict = {}
for text in root.iter('text'):
  #print(text.attrib) text ID
  for sentence in text:
    sentenceid = sentence.attrib['id']
    i = 0
    lemma_sentence = []
    surface_sentence = []
    index_targetword = []
    wordid_list = []
    pos = []
    for word in sentence:
      p = word.attrib['pos']
      lemma = word.attrib['lemma']
      surface = word.text

      try:
        wordid = word.attrib['id']
        index_targetword.append(i)

      except KeyError:
        wordid = "NA"
      surface_sentence.append(surface)
      lemma_sentence.append(lemma)
      wordid_list.append(wordid)
      pos.append(p)
      i += 1
    assert len(surface_sentence) == len(lemma_sentence)

    a = 0
    for ind in index_targetword:

     # print(lemma_sentence[ind])
      try:
        lemma_sentence_dict[lemma_sentence[ind]].append((
           " ".join(lemma_sentence),
           " ".join(surface_sentence),
           str(ind),
           str(ind),
           lemma_sentence[ind],
           wordid_list[ind] , pos[ind]
           ))
      except KeyError:
         lemma_sentence_dict[lemma_sentence[ind]]  = [(
           " ".join(lemma_sentence),
           " ".join(surface_sentence),
           str(ind),
           str(ind),
           lemma_sentence[ind],
           wordid_list[ind], pos[ind]
         )]


In [ ]:
# writing the items to a tsv file
if not os.path.exists("data_files"):
  os.mkdir("data_files")

with open(f"data_files/all.tsv", "w") as f:
  for lemma in lemma_sentence_dict:
    #f.write(lemma +"\n")
    for x in lemma_sentence_dict[lemma]:
      to_write =  "\t".join(x)+"\n"
      f.write(to_write)

In [ ]:
# loading the items from the tsv
sem_eval_list = []
with open('data_files/all.tsv', mode='r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    for row in reader:
      sentence_dict = {'sent1': row[0], 'sent2': row[1] , 'index1': row[2],
                       'index2': row[3], 'lemma': row[4], 'identifier': row[5],
                       'pos': row[6]}
      sem_eval_list.append(sentence_dict)


In [ ]:
def tok(sent):
  return sent.split(" ", 1)

In [ ]:
wordnet_id_dict = {}
with open("/content/data_files/semeval2015.gold.key.txt", mode='r') as infile:
  reader = csv.reader(infile, delimiter='\t')
  for row in reader:
    row_elems = row[0].split(" ", 1)
    # sense_dict = {'wordnet_id': row_elems[1], 'identifier': row_elems[0]}
    # labels.append(sense_dict)
    wordnet_id_dict[row_elems[0]] = row_elems[1]

FileNotFoundError: ignored

In [ ]:
wordnet_id_dict.get('d002.s006.t013')

'country%1:15:00:: country%1:14:00::'

In [ ]:
[sent_dict.update({"wordnet_id": wordnet_id_dict.get(sent_dict["identifier"])}) for sent_dict in sem_eval_list]
# [wordnet_id_dict.get(sent_dict["identifier"]) for sent_dict in sem_eval_list]
sem_eval_list

[{'sent1': 'this document be a summary of the european public assessment report ( epar ) .',
  'sent2': 'This document is a summary of the European Public Assessment Report ( EPAR ) .',
  'index1': '1',
  'index2': '1',
  'lemma': 'document',
  'identifier': 'd000.s000.t000',
  'pos': 'NOUN',
  'wordnet_id': 'document%1:10:00::'},
 {'sent1': 'the foundation aim to document the characteristic of age barrier in job recruitment and training and to analyse they cause .',
  'sent2': 'The Foundation aims to document the characteristics of age barriers in job recruitment and training and to analyse their causes .',
  'index1': '4',
  'index2': '4',
  'lemma': 'document',
  'identifier': 'd002.s015.t002',
  'pos': 'VERB',
  'wordnet_id': 'document%2:32:00::'},
 {'sent1': 'this document be a summary of the european public assessment report .',
  'sent2': 'This document is a summary of the European Public Assessment Report .',
  'index1': '1',
  'index2': '1',
  'lemma': 'document',
  'identifie

In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.10.0-py3-none-any.whl size=259296 sha256=eee380181c2be6aa7ae46f66303ee9b3192531db7f96d70e1882b5fdfec76318
  Stored in directory: /root/.cache/pip/wheels/14/9e/82/738a3a6462074384260415f8f7c3834e9309372ff627f04a2d
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase

# Prepare

In [ ]:
def execute_transaction(command):
    with GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password")) as driver:
        session = driver.session()
        print(command)
        session.run(command)
        session.close()


def execute_transactions(transaction_execution_commands):
    # data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password"))
    # session = data_base_connection.session()
    with GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password")) as driver:
        session = driver.session()
        for command in transaction_execution_commands:
            print(command)
            session.run(command)
        session.close()


def execute_query(query):
    with GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "password")) as driver:
        records, summary, keys = driver.execute_query(query)
        return records

In [ ]:
def import_to_neo4j_db(semeval_list):
    neo4j_create_commands = []
    # set of unique lemmata and sentences
    lemmata = list(set([item['lemma'] for item in semeval_list]))
    sentences = list(set([item['sent2'] for item in semeval_list]))
    wordnet_ids = list(set([item['wordnet_id'] for item in semeval_list]))

    # create nodes for all lemma, wordNet ids and sentence nodes
    for i, lemma in enumerate(lemmata):
        neo4j_create_statement = "create (l:Lemma " + "{id: " + str(i) + ", text: '" + lemma + "'})"
        neo4j_create_commands.append(neo4j_create_statement)

    for wordnet_id in wordnet_ids:
        neo4j_create_statement = "create (l:Wordnet_id " + "{id: '" + wordnet_id + "'})"
        neo4j_create_commands.append(neo4j_create_statement)

    for i, sentence in enumerate(sentences):
        if "'" not in sentence:
            sent_with_quote = f"'{sentence}'"
        else:
            sent_with_quote = f'"{sentence}"'
        neo4j_create_statement = "create (s:Sentence " + "{id: " + str(i) + ", text: " + sent_with_quote + "})"
        neo4j_create_commands.append(neo4j_create_statement)

    execute_transactions(neo4j_create_commands)

    # create relations
    for item in semeval_list:
        # create relation between lemma and sentence with pos
        sentence_id = sentences.index(item['sent2'])
        lemma_id = lemmata.index(item['lemma'])
        wordnet_id = item['wordnet_id']
        # check if HAS_SENSE relation exists
        query_statement = "MATCH (l:Lemma)-[r1:HAS_SENSE]->(wid:Wordnet_id) WHERE " \
                          f"l.id = {lemma_id} AND wid.id = '{wordnet_id}' \n" \
                          "RETURN l,wid"
        records = execute_query(query_statement)
        if len(records) == 0:
            neo4j_create_statement = "MATCH (l:Lemma), (wid:Wordnet_id), (s:Sentence) WHERE " \
                                 f"l.id = {lemma_id} AND wid.id = '{wordnet_id}' AND s.id = {sentence_id}\n" \
                                 "CREATE (l)-[:HAS_SENSE]->(wid),\n" \
                                 "(wid)-[:FOUND_IN " \
                                 "{pos:'" + item['pos'] + "', index: " + str(item['index2']) + "}]-> (s)"
        else:
            neo4j_create_statement = "MATCH (wid:Wordnet_id), (s:Sentence) WHERE " \
                                     f"wid.id = '{wordnet_id}' AND s.id = {sentence_id}\n" \
                                     "CREATE (wid)-[:FOUND_IN " \
                                     "{pos:'" + item['pos'] + "', index: " + str(item['index2']) + "}]-> (s)"

        execute_transaction(neo4j_create_statement)

In [ ]:
import_to_neo4j_db(df1.values.tolist())

# Access

In [ ]:
def get_senses_for_word(lemma):
    query = (
        "MATCH (l:Lemma {text: $lemma})-[:HAS_SENSE]->(w:wordnet_id)"
        "RETURN w"
    )
    result = execute_query(query, lemma=lemma)
    senses = [record["w"] for record in result]
    return senses


In [ ]:
def get_pos_for_lemma(lemma):
    query = (
        "MATCH (l:Lemma)-[:HAS_SENSE]->(:Wordnet_id)-[r:FOUND_IN]->(:Sentence)"
        " WHERE l.text = $lemma" # try "use" in place of $lemma
        " RETURN r.pos"
    )
    result = execute_query(query, lemma=lemma)
    pos_list = [record["r.pos"] for record in result]
    return pos_list

In [ ]:
def find_wordnet_ids_in_sentences():
  """
   Map wordnet_ids found in a sentence
  """
    query = (
        "MATCH (w:Wordnet_id)-[:FOUND_IN]->(s:Sentence)"
        "RETURN w, s"
    )
    result = execute_query(query)
    wordnet_ids = []
    sentences = []
    for record in result:
        wordnet_ids.append(record["w"])
        sentences.append(record["s"])
    return wordnet_ids, sentences

In [ ]:
def get_top_lemmas_with_wordnet_ids(limit=10):
  """
  Top 10 lemmas with highest number of wordnet_ids / senses
  """

    query = (
        "MATCH (l:Lemma)-[:HAS_SENSE]->(w:Wordnet_id)"
        "WITH l, count(w) AS numWordnetIDs"
        "ORDER BY numWordnetIDs DESC"
        "LIMIT $limit"
        "RETURN l.text AS Lemma, numWordnetIDs AS NumWordnetIDs"
    )
    result = execute_query(query, limit=limit)
    top_lemmas = []
    for record in result:
        lemma = record["Lemma"]
        num_wordnet_ids = record["NumWordnetIDs"]
        top_lemmas.append((lemma, num_wordnet_ids))
    return top_lemmas